# Pattern Generator

> Generate all valid sub-patterns from a crossword slot for word placement

The `PatternGenerator` class takes a crossword slot pattern (like `'  x ch  p'`) and generates all valid sub-patterns where words could be placed, sorted by constraint count, length, and position.

## Usage

```python
from crossword_generator.pattern_generator import PatternGenerator

# Create a pattern generator for a slot
slot = '  x ch  p'
pg = PatternGenerator(slot)

# Generate all valid patterns
patterns = pg.generate_patterns()

# Returns list of (offset, pattern) tuples
# Example: [(0, 'x ch'), (2, 'ch  p'), ...]
```

## How it works

The generator follows these rules:
1. Patterns must start at slot beginning OR have space before (for `#` marker)
2. Patterns must end at slot end OR have space after (for `#` marker)  
3. Minimum length of 2 characters
4. Must contain at least one non-space character

Patterns are sorted by:
- Constraint count (most constrained first)
- Length (longest first)
- Position type (start/end before middle)

In [ ]:
#| default_exp pattern_generator

In [ ]:
#| export
from fastcore.foundation import L

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| export
class PatternGenerator:
    def __init__(self, 
         slot_pattern:str, # the input pattern to work with  
         min_word_length=2, # minimal resulting word  length using a a subpattern 
         placeholder=' ' # character in the pattern that can be used for any letter
         ):
        self.slot_pattern = slot_pattern
        self.min_word_length = min_word_length
        self.placeholder = placeholder
    
    def generate_patterns(self):
        # Returns sorted list of (offset, pattern)
        patterns = list(self._extract_subpatterns())
        patterns.sort(key=lambda x: x[2], reverse=True)  # Sort by sort_key
        return list(L(patterns).map(lambda x: x[:2]))
    
    def _extract_subpatterns(self):
        only_spaces = all([self.placeholder == x for x in self.slot_pattern.split()])
        patterns = L()
        
        for i in range(len(self.slot_pattern) + 1 - self.min_word_length):
            for j in range(i + self.min_word_length, len(self.slot_pattern) + 1):
                part_pat = self.slot_pattern[i:j]
                
                if (i == 0 or self.slot_pattern[i-1] == self.placeholder) and \
                   (j == len(self.slot_pattern) or self.slot_pattern[j] == self.placeholder):
                    sort_key = self._make_sort_key(part_pat, i)
                    patterns.append((i, part_pat, sort_key))
        
        if not only_spaces:
            patterns = patterns.filter(lambda y: not all([self.placeholder == x for x in y[1].split()]))
        return set(patterns)
    
    def _make_sort_key(self, part_pattern, offset):
        pos_t = self._get_position_type(offset, len(part_pattern))
        cross_count = self._count_constraints(part_pattern)
        pat_len = len(part_pattern)
        return "%s_%03x_%03x" % (pos_t, cross_count, pat_len)
    
    def _count_constraints(self, pattern):
        return len(pattern.replace(self.placeholder, ''))
    
    def _get_position_type(self, offset, pattern_length):
        if offset == 0:
            return 's'
        if offset + pattern_length == len(self.slot_pattern):
            return 'l'
        return 'i'

In [ ]:
# Test with slot variable
slot = '  x ch  p'
pg = PatternGenerator(slot)
result = pg.generate_patterns()
result_patterns = set([x[1] for x in result])

assert '  x ch  p' in result_patterns
assert '  x' in result_patterns
assert '  x ch' in result_patterns
assert '  x ch ' in result_patterns
assert ' x' in result_patterns
assert ' x ch' in result_patterns
assert ' x ch ' in result_patterns
assert ' x ch  p' in result_patterns
assert 'x ch' in result_patterns
assert 'x ch ' in result_patterns
assert 'x ch  p' in result_patterns
assert 'ch' in result_patterns
assert 'ch ' in result_patterns
assert 'ch  p' in result_patterns
assert ' p' in result_patterns

# Negative tests
pg1 = PatternGenerator('x', min_word_length=2)
result1 = set([x[1] for x in pg1.generate_patterns()])
assert len(result1) == 0

pg2 = PatternGenerator('', min_word_length=2)
result2 = set([x[1] for x in pg2.generate_patterns()])
assert len(result2) == 0

pg3 = PatternGenerator('    ', min_word_length=2)
result3 = set([x[1] for x in pg3.generate_patterns()])
assert all(' ' in p for p in result3)

pg4 = PatternGenerator('abc', min_word_length=2)
result4 = set([x[1] for x in pg4.generate_patterns()])
assert len(result4) == 1

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()